In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Pre-Processing

In [93]:
user_data=pd.read_csv('application_record.csv')
record=pd.read_csv('credit_record.csv')

In [94]:
user_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [95]:
record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


## 1.1) Setting up target values to binary type

In [96]:
record['STATUS'].unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [97]:
record=record.replace({"STATUS":{'X':-1,'C':-1}})

In [98]:
record.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,-1
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,-1


In [99]:
record.dtypes

ID                 int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object

In [100]:
record['STATUS']=record['STATUS'].astype('int64')
record.dtypes

ID                int64
MONTHS_BALANCE    int64
STATUS            int64
dtype: object

In [101]:
record['STATUS']=record['STATUS']+1

In [102]:
record=record.groupby('ID').max('STATUS') 
record.head()

,MONTHS_BALANCE,STATUS
ID,,
5001711,0,1
5001712,0,1
5001713,0,0
5001714,0,0
5001715,0,0


## 1.2) Merging Record data with the Personal Data

In [103]:
record=pd.merge(user_data,record,how='inner',on='ID')
record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0,1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-22,0


# 2) Visualization 

## 2.1) Stacked Bar

### 2.1.1) Housing Type  

In [104]:
pivot_bar_housing=record.pivot_table(values='ID',index='NAME_HOUSING_TYPE',columns='STATUS',aggfunc=np.size)

In [44]:
pivot_bar_housing.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_housing

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
NAME_HOUSING_TYPE,,,,,,,
Co-op apartment,30.0,124.0,11.0,NaN,1.0,NaN,2.0
House / apartment,3967.0,24799.0,3242.0,280.0,66.0,39.0,155.0
Municipal apartment,145.0,833.0,120.0,8.0,5.0,5.0,12.0
Office apartment,34.0,190.0,29.0,5.0,2.0,NaN,2.0
Rented apartment,65.0,430.0,72.0,3.0,1.0,NaN,4.0
With parents,214.0,1335.0,201.0,18.0,1.0,2.0,5.0


In [45]:
piv_bar_excel_housing = pd.ExcelWriter('pivot_housing_vs_dues.xlsx')
pivot_bar_housing.to_excel(piv_bar_excel_housing)
piv_bar_excel_housing.save()

### 2.1.2) Education Type

In [40]:
pivot_bar_education=record.pivot_table(values='ID',index='NAME_EDUCATION_TYPE',columns='STATUS',aggfunc=np.size)
pivot_bar_education=pivot_bar_education.reindex(index=['Academic degree','Higher education','Incomplete higher','Secondary / secondary special','Lower secondary'])

In [41]:
pivot_bar_education.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_education

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
NAME_EDUCATION_TYPE,,,,,,,
Academic degree,4.0,21.0,7.0,NaN,NaN,NaN,NaN
Higher education,1265.0,7451.0,977.0,80.0,24.0,10.0,57.0
Incomplete higher,152.0,1051.0,174.0,13.0,4.0,9.0,7.0
Secondary / secondary special,2996.0,18891.0,2488.0,218.0,48.0,26.0,110.0
Lower secondary,38.0,297.0,29.0,3.0,NaN,1.0,6.0


In [46]:
piv_bar_excel_education = pd.ExcelWriter('pivot_education_vs_dues.xlsx')
pivot_bar_education.to_excel(piv_bar_excel_education)
piv_bar_excel_education.save()

### 2.1.3) Income Type

In [49]:
pivot_bar_incometype=record.pivot_table(values='ID',index='NAME_INCOME_TYPE',columns='STATUS',aggfunc=np.size)

In [50]:
pivot_bar_incometype.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_incometype

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
NAME_INCOME_TYPE,,,,,,,
Commercial associate,1096.0,6314.0,937.0,60.0,14.0,20.0,49.0
Pensioner,752.0,4756.0,514.0,59.0,26.0,7.0,38.0
State servant,349.0,2251.0,348.0,18.0,8.0,NaN,11.0
Student,NaN,10.0,1.0,NaN,NaN,NaN,NaN
Working,2258.0,14380.0,1875.0,177.0,28.0,19.0,82.0


In [52]:
piv_bar_excel_incometype = pd.ExcelWriter('pivot_incometype_vs_dues.xlsx')
pivot_bar_incometype.to_excel(piv_bar_excel_incometype)
piv_bar_excel_incometype.save()

### 2.1.4) Family Status 

In [56]:
pivot_bar_familystatus=record.pivot_table(values='ID',index='NAME_FAMILY_STATUS',columns='STATUS',aggfunc=np.size)

In [57]:
pivot_bar_familystatus.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_familystatus

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
NAME_FAMILY_STATUS,,,,,,,
Civil marriage,380.0,2198.0,321.0,38.0,2.0,NaN,6.0
Married,3061.0,19073.0,2521.0,195.0,48.0,25.0,125.0
Separated,271.0,1607.0,194.0,4.0,10.0,4.0,13.0
Single / not married,561.0,3645.0,522.0,50.0,6.0,16.0,29.0
Widow,182.0,1188.0,117.0,27.0,10.0,1.0,7.0


In [59]:
piv_bar_excel_familystatus = pd.ExcelWriter('pivot_familystatus_vs_dues.xlsx')
pivot_bar_familystatus.to_excel(piv_bar_excel_familystatus)
piv_bar_excel_familystatus.save()

### 2.1.5) Gender

In [79]:
pivot_bar_gender=record.pivot_table(values='ID',index='CODE_GENDER',columns='STATUS',aggfunc=np.size)

In [80]:
pivot_bar_gender.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_gender.index=['Female','Male']
pivot_bar_gender

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
Female,2960,18712,2379,193,54,21,111
Male,1495,8999,1296,121,22,25,69


In [67]:
piv_bar_excel_gender = pd.ExcelWriter('pivot_gender_vs_dues.xlsx')
pivot_bar_gender.to_excel(piv_bar_excel_gender)
piv_bar_excel_gender.save()

### 2.1.6) Occupation Type

In [72]:
pivot_bar_occupation=record.pivot_table(values='ID',index='OCCUPATION_TYPE',columns='STATUS',aggfunc=np.size)

In [73]:
pivot_bar_occupation.columns=['Dues Paid','Due 1 Month','Due 2 Months','Due 3 Months','Due 4 Months','Due 5 Months','Due 6 Months']
pivot_bar_occupation

,Dues Paid,Due 1 Month,Due 2 Months,Due 3 Months,Due 4 Months,Due 5 Months,Due 6 Months
OCCUPATION_TYPE,,,,,,,
Accountants,175.0,919.0,124.0,16.0,2.0,2.0,3.0
Cleaning staff,56.0,432.0,58.0,2.0,1.0,NaN,2.0
Cooking staff,80.0,489.0,77.0,4.0,NaN,NaN,5.0
Core staff,464.0,2664.0,389.0,25.0,15.0,8.0,26.0
Drivers,259.0,1615.0,215.0,34.0,3.0,5.0,7.0
HR staff,11.0,60.0,13.0,1.0,NaN,NaN,NaN
High skill tech staff,152.0,1050.0,151.0,17.0,1.0,2.0,10.0
IT staff,8.0,41.0,8.0,1.0,NaN,NaN,2.0
Laborers,812.0,4669.0,631.0,51.0,8.0,12.0,28.0


In [84]:
piv_bar_excel_occupation = pd.ExcelWriter('pivot_occupation_vs_dues.xlsx')
pivot_bar_occupation.to_excel(piv_bar_excel_occupation)
piv_bar_excel_occupation.save()

## 2.2) Pie Chart

## 2.2.1) Gender

In [87]:
pie_chart_gender=record['CODE_GENDER'].value_counts()
pie_chart_gender

F    24430
M    12027
Name: CODE_GENDER, dtype: int64

In [89]:
pie_chart_excel_gender = pd.ExcelWriter('pie_chart_gender.xlsx')
pie_chart_gender.to_excel(pie_chart_excel_gender)
pie_chart_excel_gender.save()

### 2.2.2) Status

In [90]:
pie_chart_status=record['STATUS'].value_counts()
pie_chart_status

1    27711
0     4455
2     3675
3      314
6      180
4       76
5       46
Name: STATUS, dtype: int64

In [91]:
pie_chart_excel_status = pd.ExcelWriter('pie_chart_status.xlsx')
pie_chart_status.to_excel(pie_chart_excel_status)
pie_chart_excel_status.save()

# 3) Pre-Processing for ML 

In [19]:
record['CODE_GENDER'].unique()

array(['M', 'F'], dtype=object)

In [20]:
record['FLAG_OWN_CAR'].unique()

array(['Y', 'N'], dtype=object)

In [21]:
record['FLAG_OWN_REALTY'].unique()

array(['Y', 'N'], dtype=object)

In [22]:
from function_file import change_to_bi

In [24]:
change_to_bi(record,'CODE_GENDER')
change_to_bi(record,'FLAG_OWN_CAR')
change_to_bi(record,'FLAG_OWN_REALTY')
record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,0,0,0,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
1,5008805,0,0,0,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
2,5008806,0,0,0,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
3,5008808,1,1,0,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0,1
4,5008809,1,1,0,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-22,0


In [25]:
record['NAME_EDUCATION_TYPE'].unique() # special type of categorical value

array(['Higher education', 'Secondary / secondary special',
       'Incomplete higher', 'Lower secondary', 'Academic degree'],
      dtype=object)

In [27]:
record['NAME_EDUCATION_TYPE']=record['NAME_EDUCATION_TYPE'].replace(['Lower secondary','Secondary / secondary special','Incomplete higher','Higher education','Academic degree'],[0,1,2,3,4])
record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,0,0,0,0,427500.0,Working,3,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
1,5008805,0,0,0,0,427500.0,Working,3,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0,2
2,5008806,0,0,0,0,112500.0,Working,1,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0,1
3,5008808,1,1,0,0,270000.0,Commercial associate,1,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0,1
4,5008809,1,1,0,0,270000.0,Commercial associate,1,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-22,0


In [28]:
record['NAME_INCOME_TYPE'].unique()

array(['Working', 'Commercial associate', 'Pensioner', 'State servant',
       'Student'], dtype=object)

In [29]:
record['NAME_FAMILY_STATUS'].unique()

array(['Civil marriage', 'Married', 'Single / not married', 'Separated',
       'Widow'], dtype=object)

In [30]:
record['NAME_HOUSING_TYPE'].unique()

array(['Rented apartment', 'House / apartment', 'Municipal apartment',
       'With parents', 'Co-op apartment', 'Office apartment'],
      dtype=object)

In [31]:
record['OCCUPATION_TYPE'].unique()

array([nan, 'Security staff', 'Sales staff', 'Accountants', 'Laborers',
       'Managers', 'Drivers', 'Core staff', 'High skill tech staff',
       'Cleaning staff', 'Private service staff', 'Cooking staff',
       'Low-skill Laborers', 'Medicine staff', 'Secretaries',
       'Waiters/barmen staff', 'HR staff', 'Realty agents', 'IT staff'],
      dtype=object)

In [32]:
from function_file import change_to_num # execute this code, if not installed : !pip install ipynb

In [33]:
change_to_num(record,'NAME_INCOME_TYPE')
change_to_num(record,'NAME_FAMILY_STATUS')
change_to_num(record,'NAME_HOUSING_TYPE')
change_to_num(record,'OCCUPATION_TYPE')
record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,2
1,5008805,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,2
2,5008806,0,0,0,0,112500.0,0,1,1,1,-21474,-1134,1,0,0,0,1,2.0,0,1
3,5008808,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,0,1
4,5008809,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,-22,0


In [34]:
record.loc[record['STATUS']<=0,'STATUS']=0
record.loc[record['STATUS']>0,'STATUS']=1  # on or off
record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,1
1,5008805,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,1
2,5008806,0,0,0,0,112500.0,0,1,1,1,-21474,-1134,1,0,0,0,1,2.0,0,1
3,5008808,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,0,1
4,5008809,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,-22,0


In [35]:
record[record['STATUS']==0]

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
4,5008809,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,-22,0
12,5008819,0,0,0,0,135000.0,1,1,1,1,-17778,-1194,1,0,0,0,4,2.0,-10,0
44,5008857,1,0,0,2,135000.0,0,1,1,1,-15761,-3173,1,0,0,0,4,4.0,0,0
55,5112846,1,0,0,2,135000.0,0,1,1,1,-15761,-3173,1,0,0,0,4,4.0,-15,0
76,5008893,1,1,0,0,297000.0,1,1,2,0,-15519,-3234,1,0,0,0,4,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36079,5150409,0,0,0,2,157500.0,0,1,1,1,-14312,-4480,1,1,1,0,4,4.0,0,0
36080,5150410,1,1,0,0,67500.0,2,1,4,1,-22116,365243,1,0,0,0,0,1.0,-21,0
36091,5150422,1,1,0,0,67500.0,2,1,4,1,-22116,365243,1,0,0,0,0,1.0,0,0
36097,5150460,1,0,0,1,112500.0,0,1,1,1,-14234,-309,1,0,0,0,7,3.0,0,0


In [37]:
record[record['STATUS']==1]

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,MONTHS_BALANCE,STATUS
0,5008804,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,1
1,5008805,0,0,0,0,427500.0,0,3,0,0,-12005,-4542,1,1,0,0,0,2.0,0,1
2,5008806,0,0,0,0,112500.0,0,1,1,1,-21474,-1134,1,0,0,0,1,2.0,0,1
3,5008808,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,0,1
5,5008810,1,1,0,0,270000.0,1,1,2,1,-19110,-3051,1,0,1,1,2,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,0,0,0,0,315000.0,0,1,1,1,-17348,-2420,1,0,0,0,5,2.0,0,1
36453,5149834,1,1,0,0,157500.0,1,3,1,1,-12387,-1325,1,0,1,1,13,2.0,0,1
36454,5149838,1,1,0,0,157500.0,2,3,1,1,-12387,-1325,1,0,1,1,13,2.0,0,1
36455,5150049,1,1,0,0,283500.0,0,1,1,1,-17958,-655,1,0,0,0,2,2.0,0,1
